## Case Study - Freddie Mac Bonds

In [1]:
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
# Path to Excel file containing required callable bond and discount curve data
bond_file = '../data/callable_bonds_2025-02-13.xlsx'
discount_file = '../data/discount_curve_2025-02-13.xlsx'

# Read the Excel file for info and quotes of callable bonds into a pandas DataFrame
callable_bonds_info = pd.read_excel(bond_file, sheet_name=0)
callable_bonds_quotes = pd.read_excel(bond_file, sheet_name=1)

# Read the Excel file for discount curve into a pandas DataFrame
discount_curve = pd.read_excel(discount_file, sheet_name=0)

# Display the DataFrame
display(callable_bonds_info.style.set_caption("Callable Bonds Info"))
display(callable_bonds_quotes.style.set_caption("Callable Bonds Quotes"))
display(discount_curve.head().style.set_caption("Discount Curve"))

,info,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
0,CUSIP,3134GW5F9,3134GWGK6,3134HA4V2
1,Issuer,FREDDIE MAC,FREDDIE MAC,FREDDIE MAC
2,Maturity Type,CALLABLE,CALLABLE,CALLABLE
3,Issuer Industry,GOVT AGENCY,GOVT AGENCY,GOVT AGENCY
4,Amount Issued,30000000,25000000,10000000
5,Cpn Rate,0.009700,0.012500,0.044100
6,Cpn Freq,2,2,2
7,Date Quoted,2025-02-13 00:00:00,2025-02-13 00:00:00,2025-02-13 00:00:00
8,Date Issued,2020-10-28 00:00:00,2020-07-29 00:00:00,2025-01-28 00:00:00
9,Date Matures,2028-01-28 00:00:00,2030-01-29 00:00:00,2030-01-28 00:00:00


,quotes,FHLMC 0.97 01/28/28,FHLMC 1 1/4 01/29/30,FHLMC 4.41 01/28/30
0,Date Quoted,2025-02-13 00:00:00,2025-02-13 00:00:00,2025-02-13 00:00:00
1,TTM,2.954141,4.958248,4.955510
2,Clean Price,90.144000,85.109500,99.893000
3,Dirty Price,90.187111,85.161583,100.089000
4,Accrued Interest,0.043111,0.052083,0.196000
5,YTM Call,54.240686,85.395782,4.448321
6,YTM Maturity,4.572927,4.646847,4.433845
7,Duration,2.917203,4.806058,4.496738
8,Modified Duration,2.851993,4.696929,4.399211
9,Convexity,0.095944,0.248027,0.227173


,ttm,maturity date,spot rate,discount
0,0.500000,2025-08-13 00:00:00,0.043743,0.978597
1,1.000000,2026-02-13 00:00:00,0.042890,0.958451
2,1.500000,2026-08-13 00:00:00,0.042238,0.939228
3,2.000000,2027-02-13 00:00:00,0.041843,0.920515
4,2.500000,2027-08-13 00:00:00,0.041632,0.902117


## 1. Pricing the Callable Bond

### 1.1 Bond Pricing

We will price:
1. The callable bond (FHLMC 4.41 01/28/30)
2. Hypothetical non-callable version with original maturity
3. Hypothetical non-callable version with maturity at call date

All pricing uses the provided discount curve.

In [ ]:
# Function to interpolate discount factors
def interpolate_discount_factor(ttm_target, discount_curve):
    """
    Interpolate discount factor for a given time to maturity.
    Uses linear interpolation on log of discount factors.
    """
    ttm_curve = discount_curve['ttm'].values
    discount_factors = discount_curve['discount'].values
    
    if ttm_target <= ttm_curve[0]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    elif ttm_target >= ttm_curve[-1]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    else:
        # Linear interpolation on log(discount)
        log_discounts = np.log(discount_factors)
        log_df_interpolated = np.interp(ttm_target, ttm_curve, log_discounts)
        return np.exp(log_df_interpolated)


# Generate cash flow schedule for a bond
def generate_cash_flows(date_start, date_end, coupon_rate, coupon_freq, principal=100):
    """
    Generate cash flow dates and amounts for a bond.
    
    Parameters:
    - date_start: valuation date
    - date_end: maturity date
    - coupon_rate: annual coupon rate
    - coupon_freq: coupons per year
    - principal: face value (default 100)
    
    Returns:
    - DataFrame with columns: date, time_to_cf, coupon_payment, principal_payment, total_cf
    """
    # Create coupon payment dates
    cf_dates = []
    current_date = date_end
    
    while current_date > date_start:
        cf_dates.append(current_date)
        current_date = current_date - pd.DateOffset(months=int(12/coupon_freq))
    
    cf_dates.reverse()
    
    # Calculate time to each cash flow (in years)
    times_to_cf = [(d - date_start).days / 365.25 for d in cf_dates]
    
    # Calculate coupon payments
    coupon_payment = principal * coupon_rate / coupon_freq
    coupon_payments = [coupon_payment] * len(cf_dates)
    
    # Principal payment at maturity
    principal_payments = [0] * (len(cf_dates) - 1) + [principal]
    
    # Total cash flows
    total_cfs = [c + p for c, p in zip(coupon_payments, principal_payments)]
    
    df = pd.DataFrame({
        'date': cf_dates,
        'time_to_cf': times_to_cf,
        'coupon_payment': coupon_payments,
        'principal_payment': principal_payments,
        'total_cf': total_cfs
    })
    
    return df


# Price a bond using discount curve
def price_bond(cash_flows_df, discount_curve):
    """
    Price a bond using cash flows and discount curve.
    
    Parameters:
    - cash_flows_df: DataFrame with 'time_to_cf' and 'total_cf' columns
    - discount_curve: DataFrame with 'ttm' and 'discount' columns
    
    Returns:
    - bond_price: present value of all cash flows
    """
    pv = 0
    for _, row in cash_flows_df.iterrows():
        ttm = row['time_to_cf']
        cf = row['total_cf']
        df = interpolate_discount_factor(ttm, discount_curve)
        pv += cf * df
    
    return pv    


# Extract bond information for FHLMC 4.41 01/28/30
bond_idx = 3
bond_name = 'FHLMC 4.41 01/28/30'

# Bond characteristics
coupon_rate = callable_bonds_info.iloc[5, bond_idx]
coupon_freq = callable_bonds_info.iloc[6, bond_idx]
date_quoted = pd.to_datetime(callable_bonds_info.iloc[7, bond_idx])
date_issued = pd.to_datetime(callable_bonds_info.iloc[8, bond_idx])
date_matures = pd.to_datetime(callable_bonds_info.iloc[9, bond_idx])
date_first_call = pd.to_datetime(callable_bonds_info.iloc[10, bond_idx])
date_next_call = pd.to_datetime(callable_bonds_info.iloc[11, bond_idx])
strike = callable_bonds_info.iloc[12, bond_idx]

# Market quotes
ttm = callable_bonds_quotes.iloc[1, bond_idx]
clean_price = callable_bonds_quotes.iloc[2, bond_idx]
dirty_price = callable_bonds_quotes.iloc[3, bond_idx]
accrued_interest = callable_bonds_quotes.iloc[4, bond_idx]
quoted_duration = callable_bonds_quotes.iloc[7, bond_idx]
implied_vol = callable_bonds_quotes.iloc[12, bond_idx]


# Display bond characteristics
bond_info_dict = {
    "Bond": [bond_name],
    "Coupon Rate": [f"{coupon_rate:.4f} ({coupon_rate*100:.2f}%)"],
    "Coupon Frequency": [f"{coupon_freq} (semi-annual)"],
    "Date Quoted": [date_quoted.date()],
    "Date Matures": [date_matures.date()],
    "Date Next Call": [date_next_call.date()],
    "Strike": [strike],
    "TTM (years)": [f"{ttm:.4f}"],
    "Market Dirty Price": [f"{dirty_price:.6f}"]
}

bond_info_df = pd.DataFrame(bond_info_dict)
display(bond_info_df.style.set_caption("FHLMC 4.41 01/28/30 Bond Characteristics"))

,Bond,Coupon Rate,Coupon Frequency,Date Quoted,Date Matures,Date Next Call,Strike,TTM (years),Market Dirty Price
0,FHLMC 4.41 01/28/30,0.0441 (4.41%),2 (semi-annual),2025-02-13,2030-01-28,2028-01-28,100,4.9555,100.089000


In [13]:
# 1. Market price (callable bond)
callable_bond_price = dirty_price

# 2. Hypothetical non-callable bond (full maturity)
cf_hypothetical_full = generate_cash_flows(
    date_quoted, date_matures, coupon_rate, coupon_freq,
)
price_hypothetical_full = price_bond(cf_hypothetical_full, discount_curve)

# 3. Hypothetical non-callable bond (call date maturity)
cf_hypothetical_call = generate_cash_flows(
    date_quoted, date_next_call, coupon_rate, coupon_freq,
)
price_hypothetical_call = price_bond(cf_hypothetical_call, discount_curve)

# Organize results in a DataFrame for cleaner display
pricing_summary = pd.DataFrame(
    {
        "Scenario": [
            "Market Dirty Price (Callable Bond)",
            f"Hypothetical Non-Callable (to Maturity {date_matures.date()})",
            f"Hypothetical Non-Callable (to Call Date {date_next_call.date()})",
        ],
        "Model Price": [
            f"{callable_bond_price:.6f}",
            f"{price_hypothetical_full:.6f}",
            f"{price_hypothetical_call:.6f}",
        ],
    }
)

display(pricing_summary.style.set_caption(f"Bond pricing summary - {bond_name}"))


# Market Implied Call Value = Hypothetical Non-Callable (to Maturity) - Callable Bond Price
market_implied_call_value = price_hypothetical_full - callable_bond_price

call_value_df = pd.DataFrame(
    {
        "": ["Market Implied Call Value"],
        "Value": [f"{market_implied_call_value:.6f}"],
    }
)

display(call_value_df.style.set_caption(f"Market Implied Call Value - {bond_name}"))

,Scenario,Model Price
0,Market Dirty Price (Callable Bond),100.089000
1,Hypothetical Non-Callable (to Maturity 2030-01-28),101.408472
2,Hypothetical Non-Callable (to Call Date 2028-01-28),100.902111


,,Value
0,Market Implied Call Value,1.319472


### 1.2 Forward Price of Hypothetical Bond at Call Date

Calculate the forward price of the hypothetical bond as of the call date.

The forward price is calculated as:
$$F(0,T_1) = \frac{P_{future}(T_1,T_2)}{Z(0,T_1)}$$

where:
- $P_{future}(T_1,T_2)$ = price at time $T_1$ of bond maturing at $T_2$
- $Z(0,T_1)$ = discount factor from now (time 0) to call date $T_1$
